In [ ]:
import numpy as np
import pandas as pd

In [ ]:
stocklist = pd.read_csv('C:\\Users\Dell\Downloads\selected_stocks.csv', index_col = 0)
stocklist = stocklist['stock_name']
dff = pd.DataFrame(columns= ['stockname', 'avg_profit', 'num_trades'])

In [ ]:
for stockname in stocklist:
    df_all = pd.read_csv("D:\\stock_data/"+stockname+".csv", names=('Dates', 'stockVWAP', 'futureVWAP', 'bidPrice', 'askPrice', 'total_value', 'total_size'))
    df_train = df_all[:60000]
    df_5min_train = df_train[df_train.index%5 == 0]
    df_5min_train.reset_index(drop=True, inplace=True)   
    df = df_5min_train[:12000] 
    data = np.array([])
    for i in range(224):
        data = np.append(data, df.loc[i,"stockVWAP"] ) 
    for i in range(224, 12000):
        data = np.append(data, np.mean(df.loc[i-224:i+1,"stockVWAP"])) 
    df_mean = pd.DataFrame(data)
    df["mean"] = df_mean
    df["deviation"]=0
    for ei in df.index:
        df["SPREAD"] = df.loc[ei,"askPrice"] - df.loc[ei,"bidPrice"]
        df.loc[ei,"deviation"] = df.loc[ei,"stockVWAP"]-df.loc[ei,"mean"]-df.loc[ei,"SPREAD"]-0.20
    df = df.drop(columns=["Dates","futureVWAP","bidPrice","askPrice","total_value","total_size"])
    no_of_trades = 0
    profit = 0
    for ei in df.index:
        if df.loc[ei,"deviation"] > 0:
            no_of_trades = no_of_trades+1
            profit = profit + df.loc[ei,"deviation"]
    dev_mean = profit/no_of_trades
    s = pd.Series([stockname, dev_mean, no_of_trades, profit, "1d" ],index= ['stockname', 'avg_profit', 'num_trades', 'total_profit', 'timeseries'])
    dff = dff.append(s,ignore_index=True)
    print(stockname + ":" + str(dev_mean) + ":" + str(no_of_trades) + ":" + str(profit))
dff.to_csv("profits_1d.csv")

D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

JPASSOCIAT:0.7708149574848713:3301:2544.46017465756
GMRINFRA:0.47369227868954694:1942:919.9104052151001
SUZLON:0.34528034953574654:1845:637.0422448934523
IBREALEST:4.566994136050441:6180:28224.023760791726
RELCAPITAL:11.4237738022042:6980:79737.94113938532
DLF:3.160893225021761:6206:19616.50335448505
IDFC:0.7291713955100821:3462:2524.3913712559042
SOUTHBANK:0.31376303899932295:2385:748.3248480133852
IDFCBANK:0.8774328974559319:2527:2217.27293187114
BANKBARODA:2.4267259043317493:4968:12055.97429272013
MANAPPURAM:1.9266770660044925:5076:9779.812787038803
HINDALCO:3.079160082506463:6350:19552.66652391604
TATASTEEL:7.666152020472759:6436:49339.35440376268
JINDALSTEL:2.9052943679927816:5830:16937.866165397918
RCOM:0.5907364182080611:2993:1768.0740996967268
HDIL:1.4267445907274074:5032:7179.378780540314
L_TFH:2.386379570325059:6421:15322.943221057205
NMDC:1.6725300359746715:4590:7676.912865123742
ADANIPOWER:0.7134935588743908:3385:2415.1756967898127
HCC:0.679534372812624:3048:2071.2207683328

In [ ]:
df1 = pd.read_csv("C:\\Users\Dell\profits_1d.csv")
df1.reset_index(drop=True, inplace=True)
df1

,Unnamed: 0,stockname,avg_profit,num_trades,timeseries,total_profit
0,0,JPASSOCIAT,0.770815,3301,1d,2544.460175
1,1,GMRINFRA,0.473692,1942,1d,919.910405
2,2,SUZLON,0.345280,1845,1d,637.042245
3,3,IBREALEST,4.566994,6180,1d,28224.023761
4,4,RELCAPITAL,11.423774,6980,1d,79737.941139
...,...,...,...,...,...,...
88,88,UJJIVAN,7.372043,4979,1d,36705.402606
89,89,PETRONET,4.668030,5512,1d,25730.179338
90,90,BALRAMCHIN,2.394138,5268,1d,12612.318588
91,91,BHARTIARTL,4.609856,5912,1d,27253.470011


In [ ]:
df_1 =   df1[(df1['avg_profit']>df1['avg_profit'].quantile(0.25)) &
              (df1['num_trades']>df1['num_trades'].quantile(0.25)) &
              (df1['total_profit']>df1['total_profit'].mean())] 

In [ ]:
df_1 = df_1.reset_index().drop(["index","Unnamed: 0"],axis =1)
df_1

,stockname,avg_profit,num_trades,timeseries,total_profit
0,IBREALEST,4.566994,6180,1d,28224.023761
1,RELCAPITAL,11.423774,6980,1d,79737.941139
2,TATASTEEL,7.666152,6436,1d,49339.354404
3,SUNTV,14.003456,6010,1d,84160.769859
4,RELINFRA,6.316774,6696,1d,42297.116764
5,MOTHERSUMI,4.355389,6310,1d,27482.507595
6,DHFL,6.047429,7774,1d,47012.709727
7,CANBK,5.023927,6022,1d,30254.085637
8,BHARATFIN,15.537670,6402,1d,99472.164055
9,M_MFIN,6.722883,6003,1d,40357.465607


In [ ]:
df2 = pd.read_csv("C:\\Users\Dell\profits_5d.csv")
df2.reset_index(drop=True, inplace=True)
df_2 = df2[(df2['avg_profit']>df2['avg_profit'].quantile(0.25)) &
           (df2['num_trades']>df2['num_trades'].quantile(0.25)) &
           (df2['total_profit']>df2['total_profit'].mean())] 
df_2 = df_2.reset_index().drop(["index","Unnamed: 0"],axis =1)
df_2

,stockname,avg_profit,num_trades,timeseries,total_profit
0,IBREALEST,13.125615,3552,5d,46622.186126
1,RELCAPITAL,29.384914,4267,5d,125385.428160
2,TATASTEEL,17.817701,3808,5d,67849.806168
3,SUNTV,36.783940,3133,5d,115244.083150
4,RELINFRA,13.174330,3434,5d,45240.648140
5,MOTHERSUMI,11.249984,3750,5d,42187.441128
6,DHFL,17.881528,4157,5d,74333.511532
7,CANBK,11.946114,3042,5d,36340.078405
8,BHARATFIN,35.069674,3344,5d,117272.990439
9,M_MFIN,15.509420,3311,5d,51351.689170


In [ ]:
df3 = pd.read_csv("C:\\Users\Dell\profits_14d.csv")
df3.reset_index(drop=True, inplace=True)
df_3 = df3[(df3['avg_profit']>df3['avg_profit'].quantile(0.25)) &
           (df3['num_trades']>df3['num_trades'].quantile(0.25)) &
           (df3['total_profit']>df3['total_profit'].mean())] 
df_3 = df_3.reset_index().drop(["index","Unnamed: 0"],axis =1)
df_3

,stockname,avg_profit,num_trades,timeseries,total_profit
0,IBREALEST,27.039678,713,14d,19279.290539
1,RELCAPITAL,66.046920,601,14d,39694.198869
2,TATASTEEL,35.853630,588,14d,21081.934704
3,SUNTV,77.254937,355,14d,27425.502686
4,APOLLOTYRE,17.220598,594,14d,10229.035078
5,MOTHERSUMI,28.513142,507,14d,14456.162960
6,DHFL,32.172985,659,14d,21201.997377
7,CANBK,21.554669,428,14d,9225.398162
8,BHARATFIN,55.235047,348,14d,19221.796491
9,M_MFIN,25.019931,629,14d,15737.536496


In [ ]:
df4 = pd.concat([df_1,df_2, df_3]).reset_index(drop= True)
df4

,stockname,avg_profit,num_trades,timeseries,total_profit
0,IBREALEST,4.566994,6180,1d,28224.023761
1,RELCAPITAL,11.423774,6980,1d,79737.941139
2,TATASTEEL,7.666152,6436,1d,49339.354404
3,SUNTV,14.003456,6010,1d,84160.769859
4,RELINFRA,6.316774,6696,1d,42297.116764
...,...,...,...,...,...
81,TVSMOTOR,31.008928,728,14d,22574.499256
82,KOTAKBANK,40.651077,656,14d,26667.106767
83,SRTRANSFIN,52.324159,315,14d,16482.110139
84,BHARTIARTL,18.575913,544,14d,10105.296465


In [ ]:
df4 = df4.sort_values(["total_profit"], ascending= False)
df4 = df4.reset_index(drop = True)
df4.head(40) 

,stockname,avg_profit,num_trades,timeseries,total_profit
0,INFIBEAM,62.993585,3297,5d,207689.850522
1,RELIANCE,61.537733,3135,5d,192920.791950
2,YESBANK,49.983288,3809,5d,190386.344051
3,HDFCBANK,31.236849,4799,5d,149905.639914
4,GRASIM,40.229141,3696,5d,148686.906265
5,YESBANK,20.946985,6790,1d,142230.028711
6,INFIBEAM,22.984770,5717,1d,131403.927784
7,CESC,35.432320,3561,5d,126174.490990
8,RELIANCE,21.106019,5965,1d,125897.405309
9,RELCAPITAL,29.384914,4267,5d,125385.428160


In [ ]:
thisdict = {}
a = 0

In [ ]:
for i in df4.index:
    if((df4.iloc[i,0]) in thisdict.keys()):
        pass
    else:
        thisdict[df4.iloc[i,0]]= df4.iloc[i,3]

In [ ]:
thisdict

{'INFIBEAM': '5d',
 'RELIANCE': '5d',
 'YESBANK': '5d',
 'HDFCBANK': '5d',
 'GRASIM': '5d',
 'CESC': '5d',
 'RELCAPITAL': '5d',
 'BHARATFIN': '5d',
 'SUNTV': '5d',
 'SRTRANSFIN': '1d',
 'KOTAKBANK': '5d',
 'DHFL': '5d',
 'PCJEWELLER': '5d',
 'TATASTEEL': '5d',
 'HINDPETRO': '5d',
 'AUROPHARMA': '5d',
 'TVSMOTOR': '5d',
 'BPCL': '5d',
 'INFY': '5d',
 'JUSTDIAL': '1d',
 'M_MFIN': '5d',
 'IBREALEST': '5d',
 'IOC': '5d',
 'RELINFRA': '5d',
 'BEL': '1d',
 'MOTHERSUMI': '5d',
 'BHARTIARTL': '5d',
 'GAIL': '5d',
 'ADANIPORTS': '5d',
 'UJJIVAN': '1d',
 'CANBK': '5d',
 'AXISBANK': '1d',
 'CGPOWER': '14d',
 'APOLLOTYRE': '14d'}

In [ ]:
print(len(thisdict))

34


In [ ]:
for i in range(11):
    thisdict.popitem()

In [ ]:
print(len(thisdict))

23


In [ ]:
import pandas as pd
import numpy as np
import os
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
df = pd.read_csv("D:\\stock_data/" + "ADANIPORTS" +".csv", names = ('Dates', 'stockVWAP', 'futureVWAP', 'bidPrice', 'askPrice', 'total_value', 'total_size'))
rng = pd.date_range('2017-01-02 09:16:00', periods=1000, freq='60min')
df = df.iloc[0:60000]
df = df[df.index%60 == 0]
df_new = pd.DataFrame(df['stockVWAP'].values, index = rng)

In [ ]:
df_new

,0
2017-01-02 09:16:00,268.144190
2017-01-02 10:16:00,267.523427
2017-01-02 11:16:00,268.428607
2017-01-02 12:16:00,268.722296
2017-01-02 13:16:00,269.660670
...,...
2017-02-12 20:16:00,385.177482
2017-02-12 21:16:00,385.000871
2017-02-12 22:16:00,382.004856
2017-02-12 23:16:00,385.547210


In [ ]:
temp = 0
predictions_all = None
check_empty = 1

In [ ]:
for k in range(999):                        
    df_scan = df_new.iloc[:k]
    best_aic = np.inf
        # write your arima model here
    best_order = None
    best_mdl = None
    pq_rng = range(4) # [0,1,2,3]
    d_rng = range(2) # [0,1]                        
    for i in pq_rng :
        for d in d_rng :
            for j in pq_rng :  
                try :
                    tmp_mdl = smt.ARIMA(df_scan , order=(i,d,j)).fit(trend='nc', disp = 0)
                    tmp_aic = tmp_mdl.aic
                    if tmp_aic < best_aic:
                        best_aic = tmp_aic
                        best_order = (i, d, j)
                        best_mdl = tmp_mdl
                except :
                    continue 
    if best_mdl is None:                                 
        temp = temp + 1                         
    else:
        predict = best_mdl.forecast(steps = 262, alpha = 0.01)[0]
        mean = np.mean(predict)
        threshold = np.std(predict) 
        if(df_scan.iloc[k - 1, 0] > mean + threshold):
            df_new.iloc[k, 0] = 0
        elif(df_scan.iloc[k - 1, 0] < mean - threshold):
            df_new.iloc[k, 0] = 1 
        else:
            df_new.iloc[k, 0] = 0.5      
                
predictions_new = pd.DataFrame(df_new)
predictions_new.to_csv("predictions_2w(14d).csv")
print("predictions_2w(14d).csv" + " Ready!!!")

predictions_2w(14d).csv Ready!!!
